In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline
sns.set_context('paper')
sns.set_style('darkgrid')

In [ ]:
import pymc3 as pm, theano.tensor as tt

In [ ]:
# simulate data from a known mixture distribution
np.random.seed(12345) # set random seed for reproducibility

In [ ]:
spread = 5
centers = np.array([-spread, 0, spread])
centers

In [ ]:
# simulate data from mixture distribution
k = 3
ndata = 500
v = np.random.randint(0, k, ndata)

In [ ]:
v[1:10]

In [ ]:
np.random.randn(ndata)[1:10]

In [ ]:
centers[v][1:10]

In [ ]:
data = centers[v] + np.random.randn(ndata)

plt.hist(data);

In [ ]:
# setup model
model = pm.Model()
with model:
    # cluster sizes
    p = pm.Dirichlet('p', a=np.array([1., 1., 1.]), shape=k)

    # ensure all clusters have some points
    p_min_potential = pm.Potential('p_min_potential', tt.switch(tt.min(p) < .1, -np.inf, 0))

    # cluster centers
    means = pm.Normal('means', mu=[0, 0, 0], sigma=15, shape=k)

    # break symmetry
    order_means_potential = pm.Potential('order_means_potential',
                                         tt.switch(means[1]-means[0] < 0, -np.inf, 0)
                                         + tt.switch(means[2]-means[1] < 0, -np.inf, 0))

    # measurement error
    sd = pm.Uniform('sd', lower=0, upper=20)

    # latent cluster of each observation
    category = pm.Categorical('category',
                              p=p,
                              shape=ndata)

    # likelihood for each observed value
    points = pm.Normal('obs',
                       mu=means[category],
                       sigma=sd,
                       observed=data)

In [ ]:
# fit model
with model:
    step1 = pm.Metropolis(vars=[p, sd, means])
    step2 = pm.ElemwiseCategorical(vars=[category], values=[0, 1, 2])
    tr = pm.sample(10000, step=[step1, step2], tune=5000)

In [ ]:
pm.traceplot(tr, var_names=['p', 'sd', 'means'])